In [1]:
import feedparser
import requests
from bs4 import BeautifulSoup
from config import Config
import re
# Import the os package
import os

# Import the openai package
import openai


In [3]:
url = 'http://feeds.bbci.co.uk/news/world/rss.xml'
url = 'https://feeds.a.dj.com/rss/WSJcomUSBusiness.xml'
url = 'https://stockcharts.com/articles/atom.xml'
f = feedparser.parse(url)

#cut feed down to 5 elements
f.entries = f.entries[:5]

#f.entries[0].content[0].value
# get the content via beautifulsoup from  f
text = f.entries[0].content[0].value
#get just the text from the html and remove html tags
text = re.sub('<[^<]+?>', '', text)
soup = BeautifulSoup(text, 'html.parser')
print(soup)

IndexError: list index out of range

In [ ]:
parsingType = 2

# create a list of texts
texts = []

for entry in f.entries:
    try:
        if parsingType ==1:
            #url = entry.link
            #print(entry.link)
            text = BeautifulSoup(requests.get(entry.link).text, 'html.parser').body.get_text()
            text = re.sub(r'(?<=[a-zA-Z])([A-Z])', r' \1', text).replace('  ', ' ')
            #print(text)
        elif parsingType == 2:
    
            text = entry.content[0].value
            #get just the text from the html and remove html tags
            text = re.sub('<[^<]+?>', '', text)
            text = BeautifulSoup(text, 'html.parser')
            #parse soup to text
            text = text.get_text()
            #print(text)
        texts.append(text)
    except Exception as e:
        print(e)
print(texts[0])

In this edition of StockCharts TV's The Final Bar, Dave shares how growth over value is less about the types of companies and more about the market cap represented in the largest benchmark names. He breaks down today's pop-and-drop for Bitcoin and answers questions from the newly-added live Q&A!This video originally premiered on October 16, 2023. Watch on our dedicated Final Bar page on StockCharts TV, or click this link to watch on YouTube.New episodes of The Final Bar premiere every weekday afternoon LIVE at 4pm ET. You can view all previously recorded episodes at this link.


In [23]:
#r = requests.get(url)
#use beautifulsoup to extract the text from the html
#soup = BeautifulSoup(r.text, 'html.parser')
#get only body from soup
#body = soup.body
#only get the text from the body
#text = body.get_text()

#compact version


In [24]:
#add apple, google and samsung to the each string
texts = [text + ' Apple Google Samsung' for text in texts]
texts

["In this edition of\xa0StockCharts TV's\xa0The Final Bar, Dave shares how growth over value is less about the types of companies and more about the market cap represented in the largest benchmark names. He breaks down today's pop-and-drop for Bitcoin and answers questions from the newly-added live Q&A!This video originally premiered on October 16, 2023. Watch on\xa0our dedicated\xa0Final Bar\xa0page\xa0on StockCharts TV, or click\xa0this link\xa0to watch on YouTube.New episodes of The Final Bar premiere every weekday afternoon LIVE at 4pm ET. You can view all previously recorded episodes\xa0at this link. Apple Google Samsung",
 'Waking up to a new week of geopolitical stress, fake news (Blackrock spot ETF), higher yields, softer dollar, equities rally, I thought to myself: As far as the market goes, it feels like time is standing still. Why would I think that? After all, it may not last and certainly we have opposing forces. Seasonally, Q4 should be strong. Bank earnings were solid wi

In [25]:

# From the IPython.display package, import display and Markdown
#from IPython.display import display, Markdown

# Import yfinance as yf

#import yfinance as yf

In [30]:
openai.api_key = Config.API_KEY

#create a dictionary for use in storing the data
data = {}

#loop through the texts
for i in range(len(texts)):
    prompt = """You are an assistant to a financial advisor. She tasked you with reading the text and assessing weahter it mentions a stock. Also consider if any of the stock might be impacted, if not directly mentioned
    If so provide a comma separated list of the stocks. If it does not simply respond 'no'"""

    response = openai.ChatCompletion.create(
        # = Config.API_KEY,
        model="gpt-4-0314",
        messages=[{"role": "system", "content": prompt},
                {"role": "user", "content": text}
        ])
    concludion = response.choices[0].message.content.lower()

    #check if conclusion contains yes or no
    print(concludion)
    if 'no' in concludion:
        print(0)
        print('no stocks mentioned in ' + f.entries[i].title)
    else:
        try:
            #parse the conclusion to get the list of stocks
            stocks = concludion.split(',')
            #remove any spaces and commas at beginning and end of each list entry
            stocks = [stock.strip() for stock in stocks]
            
            data[i] = {}
            #add the url to the dictionary
            data[i]['url'] = url
            #add the title to the dictionary
            data[i]['title'] = f.entries[0].title
            #add the text to the dictionary
            data[i]['text'] = text
            #add the stocks to the dictionary
            data[i]['stocks'] = stocks
        #catch any errors
        except Exception as e:
            stocks = []
            print('error ' + str(e))
        print (stocks)


no
0
no stocks mentioned in How Mega-Cap Names Dominate Through Market Cap
no
0
no stocks mentioned in When News Moves So Fast, Stocks Can Stand Still
no
0
no stocks mentioned in GNG TV: Looking at a Relief Rally or a Return to Go Trends?
no
0
no stocks mentioned in DP Trading Room: Mortgage Rates Hit Multiyear High -- Who Goes Bankrupt First?
no
0
no stocks mentioned in Sensitive Sectors Continue to Prop Up the S&P 500


In [27]:
#check if substring is in string
#substring = 'yes'
#string = concludion
#if substring in string:
#    print(1)
#else:

In [28]:
print(type(f.entries[0]))

<class 'feedparser.util.FeedParserDict'>


In [29]:
data

{}